In [13]:
import numpy as np

class DecisionTreeRootFinder:
    def __init__(self):
        pass

    def entropy(self, y):
    
        unique_labels, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    def information_gain(self, X, y, feature_idx):
    
        total_entropy = self.entropy(y)
        unique_values, value_counts = np.unique(X[:, feature_idx], return_counts=True)
        weighted_entropy = 0
        for value, count in zip(unique_values, value_counts):
            subset_indices = np.where(X[:, feature_idx] == value)[0]
            subset_entropy = self.entropy(y[subset_indices])
            weighted_entropy += (count / len(X)) * subset_entropy
        information_gain = total_entropy - weighted_entropy
        return information_gain

    def find_root_feature(self, X, y):
       
        num_features = X.shape[1]
        best_feature_idx = None
        best_information_gain = -np.inf
        for feature_idx in range(num_features):
            current_information_gain = self.information_gain(X, y, feature_idx)
            if current_information_gain > best_information_gain:
                best_information_gain = current_information_gain
                best_feature_idx = feature_idx
        return best_feature_idx
# Example usage
if __name__ == "__main__":
    X = np.array([[1, 0], [1, 1], [0, 0], [0, 1]])
    y = np.array([1,1,1,1])
    tree_root_finder = DecisionTreeRootFinder()
    root_feature_idx = tree_root_finder.find_root_feature(X, y)
    print("Root feature index:", root_feature_idx)


Root feature index: 0


In [14]:
import numpy as np

class DecisionTreeRootFinder:
    def __init__(self):
        pass

    def entropy(self, y):
    
        unique_labels, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    def information_gain(self, X, y, feature_idx):
    
        total_entropy = self.entropy(y)
        unique_values, value_counts = np.unique(X[:, feature_idx], return_counts=True)
        weighted_entropy = 0
        for value, count in zip(unique_values, value_counts):
            subset_indices = np.where(X[:, feature_idx] == value)[0]
            subset_entropy = self.entropy(y[subset_indices])
            weighted_entropy += (count / len(X)) * subset_entropy
        information_gain = total_entropy - weighted_entropy
        return information_gain

    def equal_width_binning(self, feature, num_bins):

        min_val = np.min(feature)
        max_val = np.max(feature)
        bin_width = (max_val - min_val) / num_bins
        bins = [min_val + i * bin_width for i in range(num_bins)]
        binned_feature = np.digitize(feature, bins)
        return binned_feature

    def frequency_binning(self, feature, num_bins):
        
        sorted_indices = np.argsort(feature)
        bin_size = len(feature) // num_bins
        binned_feature = np.zeros_like(feature)
        for i in range(num_bins):
            start_idx = i * bin_size
            end_idx = start_idx + bin_size if i < num_bins - 1 else len(feature)
            binned_feature[sorted_indices[start_idx:end_idx]] = i
        return binned_feature

    def find_root_feature(self, X, y, binning_type='equal_width', num_bins=None):
       
        if binning_type == 'equal_width':
            binning_function = self.equal_width_binning
        elif binning_type == 'frequency':
            binning_function = self.frequency_binning
        else:
            raise ValueError("Invalid binning type. Choose 'equal_width' or 'frequency'.")

        if num_bins is None:
            num_bins = int(np.sqrt(len(X)))  # Default number of bins

        num_features = X.shape[1]
        best_feature_idx = None
        best_information_gain = -np.inf
        for feature_idx in range(num_features):
            if len(np.unique(X[:, feature_idx])) > 1:  # Skip features with only one unique value
                binned_feature = binning_function(X[:, feature_idx], num_bins)
                current_information_gain = self.information_gain(binned_feature.reshape(-1, 1), y, 0)
                if current_information_gain > best_information_gain:
                    best_information_gain = current_information_gain
                    best_feature_idx = feature_idx
        return best_feature_idx

# Example usage
if __name__ == "__main__":
    X = np.array([[1, 0], [1, 1], [0, 0], [0, 1]])
    y = np.array([1, 0, 1, 0])

    tree_root_finder = DecisionTreeRootFinder()
    root_feature_idx = tree_root_finder.find_root_feature(X, y, binning_type='equal_width', num_bins=2)
    print("Root feature index:", root_feature_idx)


Root feature index: 1


In [15]:
import numpy as np

class TreeNode:
    def __init__(self, feature_index=None, threshold=None, value=None, left=None, right=None):
        self.feature_index = feature_index  # Index of feature to split on
        self.threshold = threshold          # Threshold value for binary split
        self.value = value                  # Class label for leaf node
        self.left = left                    # Left child node
        self.right = right                  # Right child node

class DecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def entropy(self, y):
        unique_labels, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    def information_gain(self, X, y, feature_idx, threshold):
        total_entropy = self.entropy(y)
        left_indices = np.where(X[:, feature_idx] <= threshold)[0]
        right_indices = np.where(X[:, feature_idx] > threshold)[0]
        left_entropy = self.entropy(y[left_indices])
        right_entropy = self.entropy(y[right_indices])
        weighted_entropy = (len(left_indices) / len(X)) * left_entropy + (len(right_indices) / len(X)) * right_entropy
        information_gain = total_entropy - weighted_entropy
        return information_gain

    def find_best_split(self, X, y):
        num_features = X.shape[1]
        best_information_gain = -np.inf
        best_feature_idx = None
        best_threshold = None
        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                current_information_gain = self.information_gain(X, y, feature_idx, threshold)
                if current_information_gain > best_information_gain:
                    best_information_gain = current_information_gain
                    best_feature_idx = feature_idx
                    best_threshold = threshold
        return best_feature_idx, best_threshold

    def build_tree(self, X, y, depth=0):
        if depth == self.max_depth or len(np.unique(y)) == 1:
            leaf_value = np.argmax(np.bincount(y))
            return TreeNode(value=leaf_value)
        
        feature_idx, threshold = self.find_best_split(X, y)
        if feature_idx is None:
            leaf_value = np.argmax(np.bincount(y))
            return TreeNode(value=leaf_value)
        
        left_indices = np.where(X[:, feature_idx] <= threshold)[0]
        right_indices = np.where(X[:, feature_idx] > threshold)[0]

        left_subtree = self.build_tree(X[left_indices], y[left_indices], depth+1)
        right_subtree = self.build_tree(X[right_indices], y[right_indices], depth+1)

        return TreeNode(feature_index=feature_idx, threshold=threshold, left=left_subtree, right=right_subtree)

    def fit(self, X, y):
        self.tree = self.build_tree(X, y)

    def predict_sample(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature_index] <= node.threshold:
            return self.predict_sample(x, node.left)
        else:
            return self.predict_sample(x, node.right)

    def predict(self, X):
        if self.tree is None:
            raise ValueError("Tree not fitted.")
        predictions = []
        for x in X:
            prediction = self.predict_sample(x, self.tree)
            predictions.append(prediction)
        return np.array(predictions)

if __name__ == "__main__":
    X = np.array([[1, 0], [1, 1], [0, 0], [0, 1]])
    y = np.array([1, 1, 0, 0])

    tree = DecisionTreeClassifier(max_depth=1)
    tree.fit(X, y)

    print("Tree structure:")
    print("Root feature index:", tree.tree.feature_index)
    print("Root threshold:", tree.tree.threshold)

    test_samples = np.array([[1, 0], [0, 1]])
    predictions = tree.predict(test_samples)
    print("Predictions for test samples:", predictions)


Tree structure:
Root feature index: 0
Root threshold: 0
Predictions for test samples: [1 0]
